# IIntentClassifier Tutorial
This notebook focuses on implementing and testing the `IIntentClassifier` interface.

In [ ]:
#!csharp
#nullable enable
#load "../../nlp_rag_design/SharedDefs.csx"

// Now Intent and IIntentClassifier are available to this notebook.

## The `IIntentClassifier` Interface
The `IIntentClassifier` interface defines a contract for classes that classify natural language queries into intents. Ensure the `Intent` enum is available (from shared code or the enums tutorial):
```csharp
public enum Intent { GetContactInfo, FilterByHireDate, FilterByRole, SearchByDepartment, Unknown }
```
And the interface:
```csharp
public interface IIntentClassifier
{
    (Intent intent, double confidence) Classify(string query);
}
```

## Path A – Keyword Classifier (learn the basics)
We’ll start with a very simple approach: look for keywords and map them to intents. This is great for learning because you can see the end-to-end flow without extra tooling. It’s not robust, but it helps you understand how an `IIntentClassifier` plugs into your pipeline.

In [ ]:
// Simple keyword-based classifier
public class KeywordIntentClassifier : IIntentClassifier
{
    public (Intent intent, double confidence) Classify(string query)
    {
        query = query.ToLowerInvariant();
        if (query.Contains("contact")) return (Intent.GetContactInfo, 0.9);
        if (query.Contains("hire date")) return (Intent.FilterByHireDate, 0.8);
        if (query.Contains("role")) return (Intent.FilterByRole, 0.8);
        if (query.Contains("department")) return (Intent.SearchByDepartment, 0.8);
        return (Intent.Unknown, 0.5);
    }
}

// Test the classifier
var classifier = new KeywordIntentClassifier();
var samples = new[]
{
    "Show me the department list",
    "What is Alice's contact information?",
    "List employees hired after 2020",
    "Who is the manager?" // no keywords
};

foreach (var q in samples)
{
    var (detected, score) = classifier.Classify(q);
    Console.WriteLine($"Query: '{q}' => {detected} (confidence: {score})");
}

## Path B – ML.NET Classifier (production-ready idea)
After you grasp the basics, you can train a model that learns patterns from labeled examples instead of hard-coded keywords. The next cell loads ML.NET, builds a tiny text-classification pipeline, trains on an in-memory dataset, and compares predictions to the keyword approach. In real projects, expand your dataset and persist the trained model.

In [ ]:
#r "nuget: Microsoft.ML, 3.0.1"
#r "nuget: Microsoft.ML.FastTree, 3.0.1"

using Microsoft.ML;
using Microsoft.ML.Data;

public class IntentExample
{
    public string Query { get; set; } = string.Empty;
    public string Label { get; set; } = string.Empty; // string label for training
}

public class IntentPrediction
{
    [ColumnName("PredictedLabel")] public string PredictedLabel { get; set; } = string.Empty;
    public float[] Score { get; set; } = System.Array.Empty<float>();
}

public class MlNetIntentClassifier : IIntentClassifier
{
    private readonly MLContext _ml = new MLContext(seed: 1);
    private readonly PredictionEngine<IntentExample, IntentPrediction> _engine;
    private readonly string[] _labels;

    public MlNetIntentClassifier()
    {
        // Small toy dataset – replace with your labeled data
        var data = new[]
        {
            new IntentExample { Query = "contact info for alice", Label = nameof(Intent.GetContactInfo) },
            new IntentExample { Query = "how do i contact bob", Label = nameof(Intent.GetContactInfo) },
            new IntentExample { Query = "employees hired after 2020", Label = nameof(Intent.FilterByHireDate) },
            new IntentExample { Query = "hired before feb 2021", Label = nameof(Intent.FilterByHireDate) },
            new IntentExample { Query = "list engineers", Label = nameof(Intent.FilterByRole) },
            new IntentExample { Query = "manager role", Label = nameof(Intent.FilterByRole) },
            new IntentExample { Query = "who is in marketing", Label = nameof(Intent.SearchByDepartment) },
            new IntentExample { Query = "sales department members", Label = nameof(Intent.SearchByDepartment) },
            new IntentExample { Query = "random question", Label = nameof(Intent.Unknown) },
        };

        var trainData = _ml.Data.LoadFromEnumerable(data);

        // Build pipeline: text featurization -> multiclass classifier -> map key back to string
        var pipeline = _ml.Transforms.Conversion.MapValueToKey("Label")
            .Append(_ml.Transforms.Text.FeaturizeText("Features", nameof(IntentExample.Query)))
            .Append(_ml.MulticlassClassification.Trainers.SdcaMaximumEntropy())
            .Append(_ml.Transforms.Conversion.MapKeyToValue("PredictedLabel"));

        var model = pipeline.Fit(trainData);
        _engine = _ml.Model.CreatePredictionEngine<IntentExample, IntentPrediction>(model);

        _labels = new[]
        {
            nameof(Intent.GetContactInfo),
            nameof(Intent.FilterByHireDate),
            nameof(Intent.FilterByRole),
            nameof(Intent.SearchByDepartment),
            nameof(Intent.Unknown)
        };
    }

    public (Intent intent, double confidence) Classify(string query)
    {
        var pred = _engine.Predict(new IntentExample { Query = query });
        // Map predicted string back to enum
        if (!Enum.TryParse<Intent>(pred.PredictedLabel, out var intent))
            intent = Intent.Unknown;

        // Confidence from softmax-like scores (not calibrated). Take max.
        double conf = 0.0;
        if (pred.Score != null && pred.Score.Length > 0)
        {
            var max = pred.Score.Max();
            var sum = pred.Score.Sum(s => Math.Exp(s));
            conf = Math.Exp(max) / sum;
        }
        return (intent, conf);
    }
}

// Demo both classifiers
var keywordCls = new KeywordIntentClassifier();
var mlCls = new MlNetIntentClassifier();

var testQueries = new[]
{
    "contact details for rick",
    "engineers hired after 2021",
    "who is in sales",
    "tell me something unrelated"
};

Console.WriteLine("Keyword vs ML.NET\n-------------------");
foreach (var q in testQueries)
{
    var (i1, c1) = keywordCls.Classify(q);
    var (i2, c2) = mlCls.Classify(q);
    Console.WriteLine($"Q: {q}\n  Keyword => {i1} ({c1:F2})\n  ML.NET  => {i2} ({c2:F2})\n");
}

## 5. Next Steps
1. Integrate a real NLP model (e.g., using ML.NET or an external LLM) for improved accuracy.  
2. Expand the classifier to handle more complex queries and use slot filling.  
3. Combine with `ISlotFiller` and `IQueryDispatcher` to build a full pipeline.